In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule


replace success!


In [6]:
import random
numbers = list(range(100))
random.shuffle(numbers)


In [16]:
def changeOrder(x):
  x = torch.abs(x)
  return x[:,numbers,:]

In [17]:
changeOrder_model = symbolic_trace(changeOrder)


In [18]:
changeOrder_model.print_readable()

class changeOrder(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        abs_1 = torch.abs(x);  x = None
        getitem = abs_1[(slice(None, None, None), [92, 60, 58, 53, 23, 97, 57, 95, 85, 72, 5, 38, 43, 54, 51, 2, 14, 55, 67, 33, 63, 19, 25, 40, 82, 35, 4, 77, 32, 87, 13, 10, 8, 28, 90, 94, 69, 36, 39, 42, 50, 34, 49, 31, 56, 12, 52, 70, 21, 9, 24, 48, 59, 64, 47, 20, 76, 75, 7, 26, 1, 81, 30, 88, 65, 0, 27, 99, 3, 86, 80, 93, 66, 62, 79, 68, 91, 41, 83, 6, 18, 98, 22, 45, 44, 96, 46, 78, 74, 84, 17, 61, 16, 11, 37, 15, 71, 89, 29, 73], slice(None, None, None))];  abs_1 = None
        return getitem
        


'class changeOrder(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        abs_1 = torch.abs(x);  x = None\n        getitem = abs_1[(slice(None, None, None), [92, 60, 58, 53, 23, 97, 57, 95, 85, 72, 5, 38, 43, 54, 51, 2, 14, 55, 67, 33, 63, 19, 25, 40, 82, 35, 4, 77, 32, 87, 13, 10, 8, 28, 90, 94, 69, 36, 39, 42, 50, 34, 49, 31, 56, 12, 52, 70, 21, 9, 24, 48, 59, 64, 47, 20, 76, 75, 7, 26, 1, 81, 30, 88, 65, 0, 27, 99, 3, 86, 80, 93, 66, 62, 79, 68, 91, 41, 83, 6, 18, 98, 22, 45, 44, 96, 46, 78, 74, 84, 17, 61, 16, 11, 37, 15, 71, 89, 29, 73], slice(None, None, None))];  abs_1 = None\n        return getitem\n        '

In [19]:
interp = utils.ProfilingInterpreter(changeOrder_model)
interp.run(torch.randint(low=0, high=88, size=(4096,100,32), dtype=torch.long))
print(interp.summary(True))

total true time 21.66295051574707 ms
total time: 30.247211456298828 ms
Op type        Op         Average runtime (ms)    Pct total runtime
-------------  -------  ----------------------  -------------------
call_function  getitem              11.1754               36.9469
call_function  abs_1                10.366                34.2708
output         output                0.0724792             0.239623
placeholder    x                     0.0491142             0.162376


In [5]:
changeOrder_model.graph.print_tabular()

opcode         name     target                       args                                                                   kwargs
-------------  -------  ---------------------------  ---------------------------------------------------------------------  --------
placeholder    x        x                            ()                                                                     {}
call_function  getitem  <built-in function getitem>  (x, (slice(None, None, None), [3, 4, 2, 1], slice(None, None, None)))  {}
output         output   output                       (getitem,)                                                             {}


In [4]:
def view_func(x):
  return x.view(-1,2)

def reshape_func(x):
  return x.reshape(-1,2)

In [5]:
view_model = fx.symbolic_trace(view_func)
reshape_model = fx.symbolic_trace(reshape_func)

In [6]:
print(view_model.code)




def forward(self, x):
    view = x.view(-1, 2);  x = None
    return view
    


In [11]:
view_model.graph.print_tabular()

opcode       name    target    args        kwargs
-----------  ------  --------  ----------  --------
placeholder  x       x         ()          {}
call_method  view    view      (x, -1, 2)  {}
output       output  output    (view,)     {}


In [7]:
print(reshape_model.code)




def forward(self, x):
    reshape = x.reshape(-1, 2);  x = None
    return reshape
    


In [12]:
reshape_model.graph.print_tabular()

opcode       name     target    args        kwargs
-----------  -------  --------  ----------  --------
placeholder  x        x         ()          {}
call_method  reshape  reshape   (x, -1, 2)  {}
output       output   output    (reshape,)  {}


In [8]:
def repeat(x):
  return x.repeat(4096,1,1)

In [9]:
repeat_model = fx.symbolic_trace(repeat)

In [10]:
print(repeat_model.code)




def forward(self, x):
    repeat = x.repeat(4096, 1, 1);  x = None
    return repeat
    


In [13]:
repeat_model.graph.print_tabular()

opcode       name    target    args             kwargs
-----------  ------  --------  ---------------  --------
placeholder  x       x         ()               {}
call_method  repeat  repeat    (x, 4096, 1, 1)  {}
output       output  output    (repeat,)        {}


In [ ]:
x = torch.randn(11,11,11)

x[:,]

In [16]:
def mat(x,y):
  return x @ y

In [17]:
mat_model = fx.symbolic_trace(mat)

In [19]:
mat_model.graph.print_tabular()

opcode         name    target                      args       kwargs
-------------  ------  --------------------------  ---------  --------
placeholder    x       x                           ()         {}
placeholder    y       y                           ()         {}
call_function  matmul  <built-in function matmul>  (x, y)     {}
output         output  output                      (matmul,)  {}


In [3]:
def func(type):
  def mat_func(x,y):
    return x @ y
  
  def add_func(x,y):
    return x + y
  
  if type =="mat":
    return mat_func
  else:
    return add_func

In [4]:
func_model = fx.symbolic_trace(func("mat"))

In [6]:
func_model.print_readable()

class mat_func(torch.nn.Module):
    def forward(self, x, y):
        # No stacktrace found for following nodes
        matmul = x @ y;  x = y = None
        return matmul
        


'class mat_func(torch.nn.Module):\n    def forward(self, x, y):\n        # No stacktrace found for following nodes\n        matmul = x @ y;  x = y = None\n        return matmul\n        '

In [7]:
func_model = fx.symbolic_trace(func("add"))

In [8]:
func_model.print_readable()

class add_func(torch.nn.Module):
    def forward(self, x, y):
        # No stacktrace found for following nodes
        add = x + y;  x = y = None
        return add
        


'class add_func(torch.nn.Module):\n    def forward(self, x, y):\n        # No stacktrace found for following nodes\n        add = x + y;  x = y = None\n        return add\n        '

In [13]:
def func(op):
  def _func(x,y):
    return op(x,y)
  return _func

In [14]:
import operator

In [15]:
func_model = fx.symbolic_trace(func(operator.add))

In [16]:
func_model.print_readable()

class _func(torch.nn.Module):
    def forward(self, x, y):
        # No stacktrace found for following nodes
        add = x + y;  x = y = None
        return add
        


'class _func(torch.nn.Module):\n    def forward(self, x, y):\n        # No stacktrace found for following nodes\n        add = x + y;  x = y = None\n        return add\n        '

In [17]:
def func(a,b):
  x = a + b
  y = a - b
  return x + y

In [18]:
func_model = fx.symbolic_trace(func)

In [19]:
func_model.print_readable()

class func(torch.nn.Module):
    def forward(self, a, b):
        # No stacktrace found for following nodes
        add = a + b
        sub = a - b;  a = b = None
        add_1 = add + sub;  add = sub = None
        return add_1
        


'class func(torch.nn.Module):\n    def forward(self, a, b):\n        # No stacktrace found for following nodes\n        add = a + b\n        sub = a - b;  a = b = None\n        add_1 = add + sub;  add = sub = None\n        return add_1\n        '

In [20]:
func_model.graph.print_tabular()

opcode         name    target                   args        kwargs
-------------  ------  -----------------------  ----------  --------
placeholder    a       a                        ()          {}
placeholder    b       b                        ()          {}
call_function  add     <built-in function add>  (a, b)      {}
call_function  sub     <built-in function sub>  (a, b)      {}
call_function  add_1   <built-in function add>  (add, sub)  {}
output         output  output                   (add_1,)    {}


In [21]:
def origal_pattern(x,y):
  return x + y

def replace_pattern(x,y):
  return x + 2 * y

def mathcer(name):
  def _match(match,ori,pat):
    target_node = None
    for node in pat.nodes:
      if node.op == "call_function" and node.target == operator.add:
        target_node = node
    return match.nodes_map[target_node].name == name
  return _match   

In [24]:
matches = subgraph_rewriter.replace_pattern_with_filters(func_model, origal_pattern, replace_pattern,[mathcer("add")])

In [25]:
func_model.print_readable()

class func(torch.nn.Module):
    def forward(self, a, b):
        # No stacktrace found for following nodes
        sub = a - b
        mul = 2 * b;  b = None
        add_2 = a + mul;  a = mul = None
        add_1 = add_2 + sub;  add_2 = sub = None
        return add_1
        


'class func(torch.nn.Module):\n    def forward(self, a, b):\n        # No stacktrace found for following nodes\n        sub = a - b\n        mul = 2 * b;  b = None\n        add_2 = a + mul;  a = mul = None\n        add_1 = add_2 + sub;  add_2 = sub = None\n        return add_1\n        '

In [26]:
def pow_func(x):
  return x ** 3

In [27]:
pow_func_model = symbolic_trace(pow_func)

In [28]:
pow_func_model.graph.print_tabular()

opcode         name    target                   args      kwargs
-------------  ------  -----------------------  --------  --------
placeholder    x       x                        ()        {}
call_function  pow_1   <built-in function pow>  (x, 3)    {}
output         output  output                   (pow_1,)  {}


In [34]:
def expand(x):
  return x.expand(32,1,1)

expand_func_model = symbolic_trace(expand)
expand_func_model.print_readable()

class expand(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        expand = x.expand(32, 1, 1);  x = None
        return expand
        


'class expand(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        expand = x.expand(32, 1, 1);  x = None\n        return expand\n        '

In [35]:
expand_func_model.graph.print_tabular()

opcode       name    target    args           kwargs
-----------  ------  --------  -------------  --------
placeholder  x       x         ()             {}
call_method  expand  expand    (x, 32, 1, 1)  {}
output       output  output    (expand,)      {}


In [30]:
env  = utils.get_env(pow_func_model)


In [31]:
node = env['pow_1']

In [33]:
str(node.target)

'<built-in function pow>'

In [41]:
op_map = {
  '<built-in function pow>':operator.pow
}

In [77]:
def gen_pattern_replace_and_matcher_for_single_op(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  target_node_name,
                                                  batch_size):
  env  = utils.get_env(traced)
  target_node = env[target_node_name]
  target_op = op_map[str(target_node.target)]
  
  def _match(match,ori,pat):
    target_node = None
    for node in pat.nodes:
      if node.op == "call_function" and node.target == target_op:
        target_node = node
    return match.nodes_map[target_node].name == target_node_name
  
  def pattern(x,y):
    return target_op(x,y)
  
  def replace(x,y):
    redency_part = x[redency_part_slice]
    unredency_part = x[unredency_part_slice]
    redency_part_result = torch.unsqueeze(target_op(redency_part,y),0).expand(batch_size,-1,-1)
    unredency_part_result = target_op(unredency_part,y)
    return torch.concat((redency_part_result,unredency_part_result),1)
  
  return pattern,replace,_match


In [78]:
def pow_test_func(x):
  x = x ** 2
  return x + 1

In [79]:
pow_test_model_func = symbolic_trace(pow_test_func)
pow_test_model_func.graph.print_tabular()

opcode         name    target                   args        kwargs
-------------  ------  -----------------------  ----------  --------
placeholder    x       x                        ()          {}
call_function  pow_1   <built-in function pow>  (x, 2)      {}
call_function  add     <built-in function add>  (pow_1, 1)  {}
output         output  output                   (add,)      {}


In [80]:
modify_mod = GraphModule(pow_test_model_func, pow_test_model_func.graph)
torch.onnx.export(modify_mod,torch.ones((4096,22,8)),f'modify_mod_pow.onnx')

In [81]:
pattern,replace,match = gen_pattern_replace_and_matcher_for_single_op(pow_test_model_func,
                                                                      (0,slice(None,10,None)),(slice(None,None,None),slice(10,None,None)),
                                                                      "pow_1",4096)


In [82]:
matches = subgraph_rewriter.replace_pattern_with_filters(pow_test_model_func, pattern, replace,[match])

In [83]:
pow_test_model_func.graph.print_tabular()

opcode         name       target                                                        args                                                   kwargs
-------------  ---------  ------------------------------------------------------------  -----------------------------------------------------  --------
placeholder    x          x                                                             ()                                                     {}
call_function  getitem    <built-in function getitem>                                   (x, (0, slice(None, 10, None)))                        {}
call_function  getitem_1  <built-in function getitem>                                   (x, (slice(None, None, None), slice(10, None, None)))  {}
call_function  pow_2      <built-in function pow>                                       (getitem, 2)                                           {}
call_function  unsqueeze  <built-in method unsqueeze of type object at 0x7f9c7895e760>  (pow_2, 0)                

In [84]:
modify_mod = GraphModule(pow_test_model_func, pow_test_model_func.graph)
torch.onnx.export(modify_mod,torch.ones((4096,22,8)),f'modify_mod_pow_m.onnx')

In [104]:
def reduce_func(x):
  x = x  + 1
  return torch.sum(x,1)

reduce_func_model = symbolic_trace(reduce_func)

In [94]:
reduce_func_model.graph.print_tabular()

opcode         name    target                                                  args      kwargs
-------------  ------  ------------------------------------------------------  --------  --------
placeholder    x       x                                                       ()        {}
call_function  add     <built-in function add>                                 (x, 1)    {}
call_function  sum_1   <built-in method sum of type object at 0x7f9c7895e760>  (add, 1)  {}
output         output  output                                                  (sum_1,)  {}


In [102]:
def pattern_func(x):
  return torch.sum(x,1)

def replace_func(x):
  x = x * 2
  return torch.sum(x,1)

matches = subgraph_rewriter.replace_pattern_with_filters(reduce_func_model, pattern_func, replace_func,[])

In [103]:
reduce_func_model.graph.print_tabular()

opcode         name    target                                                  args      kwargs
-------------  ------  ------------------------------------------------------  --------  --------
placeholder    x       x                                                       ()        {}
call_function  add     <built-in function add>                                 (x, 1)    {}
call_function  mul     <built-in function mul>                                 (add, 2)  {}
call_function  sum_2   <built-in method sum of type object at 0x7f9c7895e760>  (mul, 1)  {}
output         output  output                                                  (sum_2,)  {}


In [105]:
def gen_pattern_replace_and_matcher_for_reduce_sum(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  target_node_name
                                                ):
  env  = utils.get_env(traced)
  target_node = env[target_node_name]
  target_args = target_node.args
  target_op = torch.sum
  def _match(match,ori,pat):
    target_node = None
    for node in pat.nodes:
      if node.op == "call_function" and node.target == target_op:
        target_node = node
    return match.nodes_map[target_node].name == target_node_name
  
  def pattern(x):
    return target_op(x,target_args[1])
  
  def replace(x):
    redency_part = x[redency_part_slice]
    unredency_part = x[unredency_part_slice]
    redency_part_result = target_op(torch.unsqueeze(redency_part,0),target_args[1])
    unredency_part_result = target_op(unredency_part,target_args[1])
    return redency_part_result + unredency_part_result
  
  return pattern,replace,_match

In [106]:
pattern,replace,match = gen_pattern_replace_and_matcher_for_reduce_sum(reduce_func_model,
                                                                      (0,slice(None,10,None)),(slice(None,None,None),slice(10,None,None)),
                                                                      "sum_1")


In [107]:
matches = subgraph_rewriter.replace_pattern_with_filters(reduce_func_model, pattern, replace,[match])

In [108]:
reduce_func_model.graph.print_tabular()

opcode         name       target                                                        args                                                     kwargs
-------------  ---------  ------------------------------------------------------------  -------------------------------------------------------  --------
placeholder    x          x                                                             ()                                                       {}
call_function  add        <built-in function add>                                       (x, 1)                                                   {}
call_function  getitem    <built-in function getitem>                                   (add, (0, slice(None, 10, None)))                        {}
call_function  getitem_1  <built-in function getitem>                                   (add, (slice(None, None, None), slice(10, None, None)))  {}
call_function  unsqueeze  <built-in method unsqueeze of type object at 0x7f9c7895e760>  (getitem, 0)  

In [ ]:
def gen_pattern_replace_and_matcher_for_linear(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  target_node_name
                                                ):

In [269]:
class TestModel(torch.nn.Module):
  
  def __init__(self, input_dim, output_dim):
    super().__init__()
    self.mlp = torch.nn.Linear(input_dim, output_dim)
    self.mlp2 = torch.nn.Linear(output_dim,1)
    self.sigmoid = torch.nn.Sigmoid()
    self.mlp2.weight.data.copy_(torch.zeros_like(self.mlp2.weight.data))
    
  def forward(self,x):
    return self.sigmoid(self.mlp2(self.mlp(x)))

In [270]:
linear_func_model = symbolic_trace(TestModel(300,100))
linear_func_model.graph.print_tabular()

opcode       name     target    args        kwargs
-----------  -------  --------  ----------  --------
placeholder  x        x         ()          {}
call_module  mlp      mlp       (x,)        {}
call_module  mlp2     mlp2      (mlp,)      {}
call_module  sigmoid  sigmoid   (mlp2,)     {}
output       output   output    (sigmoid,)  {}


In [228]:
sample = torch.ones((4096,300))
modify_mod = GraphModule(linear_func_model, linear_func_model.graph)

torch.onnx.export(modify_mod,sample,f'linear.onnx')

In [145]:
mlp2 = utils.get_target_mod(linear_func_model,'mlp2')

In [146]:
mlp2.weight.data

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]])

In [271]:
class PatternClass(torch.nn.Module):
  def __init__(self, input_dim, output_dim):
    super().__init__()
    self.mlp2 = torch.nn.Linear(input_dim, 1)
    # self.mlp2 = torch.nn.Linear(output_dim * 2,1)
  def forward(self,x):
    return self.mlp2(x)
  
  
class ReplacementClass(torch.nn.Module):
  def __init__(self, input_dim, output_dim):
    super().__init__()
    self.mlp1_1 = torch.nn.Linear(input_dim, output_dim)
  def forward(self,x):
    return self.mlp1_1(x + 1)

In [272]:
subgraph_rewriter.replace_pattern(linear_func_model, PatternClass(300,100), ReplacementClass(300,100))

[Match(anchor=mlp2, nodes_map={mlp2: mlp2, x: mlp})]

In [245]:
dir(linear_func_model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__jit_unused_properties__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_deploy__',
 '__reduce_ex__',
 '__reduce_package__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_code',
 '_compiled_call_impl',
 '_deepcopy_init',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_graph',
 '_is_full_backward_hook',
 '_lineno_map',
 '_load_from_state_dict',
 

In [253]:
mlp2 = utils.get_target_mod(linear_func_model,'mlp2_1')

In [254]:
mlp2.weight.data

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [264]:
linear_func_model.graph.print_tabular()

opcode         name     target                   args        kwargs
-------------  -------  -----------------------  ----------  --------
placeholder    x        x                        ()          {}
call_function  add      <built-in function add>  (x, 1)      {}
call_module    mlp1_1   mlp1_1                   (add,)      {}
call_module    mlp2     mlp2                     (mlp1_1,)   {}
call_module    sigmoid  sigmoid                  (mlp2,)     {}
output         output   output                   (sigmoid,)  {}


In [222]:
linear_func_model.print_readable()

class TestModel(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        add = x + 1;  x = None
        mlp1_1 = self.mlp1_1(add);  add = None
        mlp2_1 = self.mlp2_1(mlp1_1);  mlp1_1 = None
        sigmoid = self.sigmoid(mlp2_1);  mlp2_1 = None
        return sigmoid
        


'class TestModel(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        add = x + 1;  x = None\n        mlp1_1 = self.mlp1_1(add);  add = None\n        mlp2_1 = self.mlp2_1(mlp1_1);  mlp1_1 = None\n        sigmoid = self.sigmoid(mlp2_1);  mlp2_1 = None\n        return sigmoid\n        '

In [223]:
mlp2 = utils.get_target_mod(linear_func_model,'mlp2_1')

In [224]:
mlp2.weight.data.shape

torch.Size([100, 100])

In [240]:
sample = torch.ones((4096,300))
modify_mod = GraphModule(linear_func_model, linear_func_model.graph)

torch.onnx.export(modify_mod,sample,f'linear_m.onnx')

In [279]:
class TestModel(torch.nn.Module):
  
  def __init__(self, input_dim, output_dim):
    super().__init__()
    self.mlp1 = torch.nn.Linear(input_dim, output_dim)
    self.mlp2 = torch.nn.Linear(output_dim,1)
    
  def forward(self,x):
    return self.mlp2(self.mlp1(x))

linear_func_model = symbolic_trace(TestModel(300,100))
linear_func_model.print_readable()

class TestModel(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        mlp1 = self.mlp1(x);  x = None
        mlp2 = self.mlp2(mlp1);  mlp1 = None
        return mlp2
        


'class TestModel(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        mlp1 = self.mlp1(x);  x = None\n        mlp2 = self.mlp2(mlp1);  mlp1 = None\n        return mlp2\n        '

In [280]:
class PatternClass(torch.nn.Module):
  def __init__(self, input_dim, output_dim):
    super().__init__()
    self.mlp1 = torch.nn.Linear(input_dim, 1)
  def forward(self,x):
    return self.mlp1(x)
  
  
class ReplacementClass(torch.nn.Module):
  def __init__(self, input_dim, output_dim):
    super().__init__()
    self.mlp1 = torch.nn.Linear(input_dim, output_dim*2)
    self.mlp3 = torch.nn.Linear(output_dim * 2,output_dim)
  def forward(self,x):
    return self.mlp3(self.mlp1(x + 1))

In [281]:
subgraph_rewriter.replace_pattern(linear_func_model, PatternClass(300,100), ReplacementClass(300,100))

[Match(anchor=mlp1, nodes_map={mlp1: mlp1, x: x})]

In [282]:
linear_func_model.print_readable()

class TestModel(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        add = x + 1;  x = None
        mlp1_1 = self.mlp1(add);  add = None
        mlp3 = self.mlp3(mlp1_1);  mlp1_1 = None
        mlp2 = self.mlp2(mlp3);  mlp3 = None
        return mlp2
        


'class TestModel(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        add = x + 1;  x = None\n        mlp1_1 = self.mlp1(add);  add = None\n        mlp3 = self.mlp3(mlp1_1);  mlp1_1 = None\n        mlp2 = self.mlp2(mlp3);  mlp3 = None\n        return mlp2\n        '

In [283]:
target_modlue = utils.get_target_mod(linear_func_model,'mlp2')

In [284]:
type(target_modlue)

torch.nn.modules.linear.Linear

In [294]:
from torch.fx import (
    Node,
    Graph,
)
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher
def new_nodes_are_equal(self, pn: Node, gn: Node) -> bool:
    # if exact match for placeholder is not required, then use placeholder as a wildcard
    if not self.match_placeholder and pn.op == "placeholder":
        return True

    if pn.op == gn.op:
        if pn.op == "placeholder" or pn.op == "output":
            return True
        elif pn.op == "get_attr":
            return self._match_attributes(pn, gn)
        elif pn.op == "call_module":
          pn_module = utils.get_target_mod(pn.graph.owning_module,pn.target)
          gn_module = utils.get_target_mod(gn.graph.owning_module,pn.target)
          return type(pn_module) == type(gn_module)
        return pn.target == gn.target
    return False

In [288]:
ori_func = SubgraphMatcher._nodes_are_equal

In [295]:
SubgraphMatcher._nodes_are_equal = new_nodes_are_equal

In [296]:
class TestModel(torch.nn.Module):
  
  def __init__(self, input_dim, output_dim):
    super().__init__()
    self.mlp1 = torch.nn.Linear(input_dim, output_dim)
    self.mlp2 = torch.nn.Linear(output_dim,1)
    
  def forward(self,x):
    return self.mlp2(self.mlp1(x))

linear_func_model = symbolic_trace(TestModel(300,100))
linear_func_model.print_readable()

class TestModel(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        mlp1 = self.mlp1(x);  x = None
        mlp2 = self.mlp2(mlp1);  mlp1 = None
        return mlp2
        


'class TestModel(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        mlp1 = self.mlp1(x);  x = None\n        mlp2 = self.mlp2(mlp1);  mlp1 = None\n        return mlp2\n        '

In [297]:
class PatternClass(torch.nn.Module):
  def __init__(self, input_dim, output_dim):
    super().__init__()
    self.mlp1 = torch.nn.Linear(input_dim, 1)
  def forward(self,x):
    return self.mlp1(x)
  
  
class ReplacementClass(torch.nn.Module):
  def __init__(self, input_dim, output_dim):
    super().__init__()
    self.mlp1 = torch.nn.Linear(input_dim, output_dim*2)
    self.mlp3 = torch.nn.Linear(output_dim * 2,output_dim)
  def forward(self,x):
    return self.mlp3(self.mlp1(x + 1))

In [298]:
subgraph_rewriter.replace_pattern(linear_func_model, PatternClass(300,100), ReplacementClass(300,100))

[Match(anchor=mlp1, nodes_map={mlp1: mlp1, x: x}),
 Match(anchor=mlp1, nodes_map={mlp1: mlp2, x: mlp3})]

In [299]:
linear_func_model.print_readable()

class TestModel(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        add = x + 1;  x = None
        mlp1_1 = self.mlp1(add);  add = None
        mlp3 = self.mlp3(mlp1_1);  mlp1_1 = None
        add_1 = mlp3 + 1;  mlp3 = None
        mlp1_2 = self.mlp1(add_1);  add_1 = None
        mlp3_1 = self.mlp3(mlp1_2);  mlp1_2 = None
        return mlp3_1
        


'class TestModel(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        add = x + 1;  x = None\n        mlp1_1 = self.mlp1(add);  add = None\n        mlp3 = self.mlp3(mlp1_1);  mlp1_1 = None\n        add_1 = mlp3 + 1;  mlp3 = None\n        mlp1_2 = self.mlp1(add_1);  add_1 = None\n        mlp3_1 = self.mlp3(mlp1_2);  mlp1_2 = None\n        return mlp3_1\n        '